# <center>Machine Learning Project</center>

** **
## <center>*03.10 - Gradient Boost*</center>

** **

The members of the `team` are:
- Ana Farinha - 20211514
- Francisco Capontes - 20211692
- Sofia Gomes - 20240848
- Rui Lourenço - 2021639



In [1]:
# Import libraries
import pandas as pd
import numpy as np

#make the split here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold
import time

from utils import *
from utils_feature_selection import check_performace
from utils_dicts import *

import warnings
warnings.filterwarnings('ignore')

## <span style="color:salmon"> 1. Import Dataset </span> 

In [2]:
# Import dataset
train_df = pd.read_csv('preprocessed_data/train_data.csv', index_col="Claim Identifier")

,Accident Date,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Known Accident Date,Known Assembly Date,Known C-2 Date,Known C-3 Date,...,Zip_Code_Hash_13,Zip_Code_Hash_14,Zip_Code_Hash_15,Zip_Code_Hash_16,Zip_Code_Hash_17,Zip_Code_Hash_18,Zip_Code_Hash_19,Claim Injury Type Encoded,Accident_Season_Sin,Accident_Season_Cos
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5393875,2019-12-30,31.0,NaN,1988.0,0.0,1.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000e+00,1.000000e+00
5393091,2019-08-30,46.0,1745.93,1973.0,4.0,4.0,1,1,1,1,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,3,1.224647e-16,-1.000000e+00
5393889,2019-12-06,40.0,1434.80,1979.0,0.0,6.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,3,-1.000000e+00,-1.836970e-16
5393887,2019-12-30,61.0,NaN,1958.0,0.0,1.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000e+00,1.000000e+00
5393863,2019-12-26,67.0,NaN,1952.0,0.0,5.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.000000e+00,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6165265,2022-12-14,35.0,NaN,1987.0,0.0,4.0,1,1,1,0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1,-1.000000e+00,-1.836970e-16
6165285,2022-12-15,33.0,NaN,1989.0,0.0,6.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,-1.000000e+00,-1.836970e-16
6165506,2022-12-13,61.0,991.08,1961.0,1.0,3.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,3,-1.000000e+00,-1.836970e-16


In [3]:
# Import dataset
test_df = pd.read_csv('./preprocessed_data/test_data.csv', index_col = 'Claim Identifier')

,Accident Date,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Known Accident Date,Known Assembly Date,Known C-2 Date,Known C-3 Date,...,Zip_Code_Hash_12,Zip_Code_Hash_13,Zip_Code_Hash_14,Zip_Code_Hash_15,Zip_Code_Hash_16,Zip_Code_Hash_17,Zip_Code_Hash_18,Zip_Code_Hash_19,Accident_Season_Sin,Accident_Season_Cos
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
6165911,2022-12-24,19.0,NaN,2003.0,0.0,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.000000e+00
6166141,2022-11-20,19.0,NaN,2003.0,0.0,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000000e+00,-1.836970e-16
6165907,2022-12-26,59.0,NaN,1963.0,0.0,0,1,1,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000e+00,1.000000e+00
6166047,2022-12-28,55.0,NaN,1967.0,0.0,6,1,1,1,0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.000000e+00
6166102,2022-12-20,25.0,NaN,1997.0,0.0,5,1,1,1,0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.000000e+00,-1.836970e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6553137,2012-09-12,52.0,NaN,1960.0,0.0,5,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.224647e-16,-1.000000e+00
6553119,2024-05-22,59.0,NaN,1965.0,0.0,1,1,1,0,1,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.000000e+00,6.123234e-17
6553542,2024-05-06,45.0,NaN,1979.0,0.0,5,1,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.000000e+00,6.123234e-17


## <span style="color:salmon"> 2. Prepare Dataset </span> 

Define y as a target "Claim Injury Type Encoded" and X with all the other columns

In [4]:
X = train_df.drop(["Claim Injury Type Encoded"], axis = 1)
y = train_df["Claim Injury Type Encoded"]

## <span style="color:salmon"> 3. Gradient Boosting</span> 

In [13]:
config = {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 100}

In [14]:
model = GradientBoostingClassifier(
        max_depth=config["max_depth"],
        learning_rate=config["learning_rate"],
        n_estimators=config["n_estimators"],
        verbose = 0
    )

GradientBoostingClassifier(max_depth=5)

In [ ]:
check_performace(model,X,y,numerical_features,essential_features,n_folds = 5)

In [ ]:
check_performace(model,X,y,numerical_features,reduced_features,n_folds = 5)

In [ ]:
check_performace(model,X,y,numerical_features,[],n_folds = 5)

#### <span style="color:salmon"> 3.1  Evaluate the model </span> 


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.75, stratify = y, shuffle = True)

In [ ]:
X_train, X_val = apply_frequency_encoding(X_train, X_val)
NA_imputer(X_train,X_val)
create_new_features(X_train,X_val)

In [ ]:
scaler = StandardScaler().fit(X_train[numerical_features])
X_train[features_to_scale]  = scaler.transform(X_train[numerical_features])
X_val[features_to_scale]  = scaler.transform(X_val[numerical_features])  

In [ ]:
drop_list = []
for col in X_train.columns:
    if col not in feature_selection:
        drop_list.append(col)
X_train = X_train.drop(drop_list, axis=1)
X_val = X_val.drop(drop_list, axis=1)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)

In [ ]:
y_val_pred = model.predict(X_val)

In [17]:
class_mapping = {
    0:'1. CANCELLED', 
    1:'2. NON-COMP',
    2:'3. MED ONLY', 
    3:'4. TEMPORARY',
    4:'5. PPD SCH LOSS', 
    5:'6. PPD NSL', 
    6:'7. PTD', 
    7:'8. DEATH'
}

# Use the values from class_mapping as the target names
target_names = list(class_mapping.values())

Compute confusion matrix to evaluate the accuracy of a classification

In [18]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, target_names=target_names))

Confusion Matrix:
[[ 1497  1518    47    53     2     1     0     1]
 [  323 71592   359   460    29     0     1     5]
 [   15  9210  1587  5785   618     0     3     9]
 [   15  2133   559 32870  1527     8     2    13]
 [    1   213   177  5111  6564     2     1     1]
 [    0     1     7   710   110   224     0     1]
 [    0     0     0    13     3     0     8     0]
 [    0    10     4    43     1     0     0    59]]

Classification Report:
                 precision    recall  f1-score   support

   1. CANCELLED       0.81      0.48      0.60      3119
    2. NON-COMP       0.85      0.98      0.91     72769
    3. MED ONLY       0.58      0.09      0.16     17227
   4. TEMPORARY       0.73      0.89      0.80     37127
5. PPD SCH LOSS       0.74      0.54      0.63     12070
     6. PPD NSL       0.95      0.21      0.35      1053
         7. PTD       0.53      0.33      0.41        24
       8. DEATH       0.66      0.50      0.57       117

       accuracy                   

In [19]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=target_names))


Confusion Matrix:
[[  3541   5417    184    192     10      3      0      9]
 [  1747 213583   1222   1584     83      9      3     78]
 [    72  27782   3709  18039   1987     40      8     42]
 [   104   6393   2079  96647   5734    285     36    102]
 [     6    681    619  16422  18369     79     13     21]
 [     0      8     52   2562    487     38      4      7]
 [     0      0      2     61      8      0      0      2]
 [     5     59     20    228     11      0      0     30]]

Classification Report:
                 precision    recall  f1-score   support

   1. CANCELLED       0.65      0.38      0.48      9356
    2. NON-COMP       0.84      0.98      0.90    218309
    3. MED ONLY       0.47      0.07      0.12     51679
   4. TEMPORARY       0.71      0.87      0.78    111380
5. PPD SCH LOSS       0.69      0.51      0.58     36210
     6. PPD NSL       0.08      0.01      0.02      3158
         7. PTD       0.00      0.00      0.00        73
       8. DEATH       0.10  

## <span style="color:salmon"> 4. Test Predictions </span> 

Make validation predictions:

In [ ]:
X, X_test = apply_frequency_encoding(X, X_test)
NA_imputer(X, X_test)
create_new_features(X, X_test)

In [ ]:
scaler = StandardScaler().fit(X[numerical_features])
X[features_to_scale]  = scaler.transform(X[numerical_features])
X_test[features_to_scale]  = scaler.transform(X_test[numerical_features])  

In [ ]:
drop_list = []
for col in X.columns:
    if col not in feature_selection:
        drop_list.append(col)
X_test = X_test.drop(drop_list, axis=1)

In [20]:
# Make validation predictions
y_test_pred = model.predict(X_test)
y_test_pred = y_test_pred.ravel()

In [ ]:
y_test_final = np.array([class_mapping[i] for i in y_test_pred])

In [22]:
test_id = model.index

AttributeError: 'GradientBoostingClassifier' object has no attribute 'index'

In [ ]:
submission_df = pd.DataFrame({
    'Claim Identifier': test_id,
    'Claim Injury Type': y_test_final
})

In [ ]:
if False:
    version = version_control()
    submission_df.to_csv(f'./submissions/Group49_Version{version:02}.csv', index=False)